In [2]:
#train the baseline CNN model for MNIST. Model is identical to that used in the tensorflow
#tutorial: https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html

import mnist_input
# import mnist_model
# reload(mnist_model)
import numpy as np
import tensorflow as tf
mnist = mnist_input.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [29]:
import mnist_model
from importlib import reload

reload(mnist_model)

<module 'mnist_model' from '/Users/xin/Desktop/BU/Study/CS591/project/Robust-CNN-with-Differential-Privacy/my_cnn/mnist_model.py'>

In [30]:
# checkpoint_path = "save_models/baseline_40epochs.ckpt"
checkpoint_path = "save_models/baseline/"
x, y_ = mnist_model.place_holders()
y_conv, keep_prob, variable_dict = mnist_model.model(x)

In [31]:
variable_dict

{'W_conv1': <tf.Variable 'W_conv1_5:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 'b_conv1': <tf.Variable 'b_conv1_5:0' shape=(32,) dtype=float32_ref>,
 'W_conv2': <tf.Variable 'W_conv2_5:0' shape=(5, 5, 32, 64) dtype=float32_ref>,
 'b_conv2': <tf.Variable 'b_conv2_5:0' shape=(64,) dtype=float32_ref>,
 'W_fc1': <tf.Variable 'W_fc1_5:0' shape=(3136, 1024) dtype=float32_ref>,
 'b_fc1': <tf.Variable 'b_fc1_5:0' shape=(1024,) dtype=float32_ref>,
 'W_fc2': <tf.Variable 'W_fc2_5:0' shape=(1024, 10) dtype=float32_ref>,
 'b_fc2': <tf.Variable 'b_fc2_5:0' shape=(10,) dtype=float32_ref>}

In [32]:
x.get_shape()[-1]

Dimension(784)

In [33]:
cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10,1.0))) #avoid 0*log(0) error

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

saver = tf.train.Saver(variable_dict)
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
# saver.restore(sess, checkpoint_path)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [34]:
#define the l1 norm of all model parameters
absolute_sums = []
for variable in variable_dict.values():
    absolute_sums.append(tf.reduce_sum(tf.abs(variable)))
l1_sum = tf.add_n(absolute_sums)

In [35]:
#compute accuracy on test set
#my GPU gives a memory error if I try to run accuracy on the whole test set
#so we feed in the test set in batches and aggregate the results
def print_test_accuracy():
    idx = 0
    batch_size = 500
    num_correct = 0
    while(idx < len(mnist.test.images)):
        num_correct += np.sum(correct_prediction.eval(feed_dict = {
               x: mnist.test.images[idx:idx+batch_size], 
               y_: mnist.test.labels[idx:idx+batch_size], keep_prob: 1.0
                    }))
        idx+=batch_size
    print ("test accuracy: %f" %(float(num_correct)/float(len(mnist.test.images))))


In [37]:
# for i in range(70000):
for i in range(7000):
    batch = mnist.train.next_batch(50)
#     if i%1000 == 0: #every 1000 batches we save the model and output the train accuracy
    if i%100 == 0:
        print_test_accuracy()
        saver.save(sess, checkpoint_path)
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        l1_norm = sess.run(l1_sum)
        print("step %d, training accuracy %g, l1_sum: %f"%(i, train_accuracy, l1_norm))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})




test accuracy: 0.106400
step 0, training accuracy 0.06, l1_sum: 236729.031250
test accuracy: 0.088200
step 100, training accuracy 0.12, l1_sum: 236728.281250
test accuracy: 0.100600
step 200, training accuracy 0.04, l1_sum: 236728.421875
test accuracy: 0.125200
step 300, training accuracy 0.1, l1_sum: 236728.187500
test accuracy: 0.144000
step 400, training accuracy 0.16, l1_sum: 236727.828125
test accuracy: 0.162100
step 500, training accuracy 0.1, l1_sum: 236727.531250
test accuracy: 0.175500
step 600, training accuracy 0.14, l1_sum: 236727.437500


KeyboardInterrupt: 

In [6]:
#total weight of all of the parameters in the model (for comparison with the l1-regularized model)
sess.run(l1_sum)

239372.95